In [63]:
# Libs
import numpy as np
import pandas as pd
import os

SENATE_CSV = '../raw_data/senate_results/1976-2018-senate.csv'

In [64]:
df = pd.read_csv(SENATE_CSV, delimiter=',', encoding='utf-8')
df.head()

,year,state,state_po,state_fips,state_cen,state_ic,office,district,stage,special,candidate,party,writein,mode,candidatevotes,totalvotes,unofficial,version
0,1976,Arizona,AZ,4,86,61,US Senate,statewide,gen,False,Sam Steiger,republican,False,total,321236,741210,False,20171011.0
1,1976,Arizona,AZ,4,86,61,US Senate,statewide,gen,False,Wm. Mathews Feighan,independent,False,total,1565,741210,False,20171011.0
2,1976,Arizona,AZ,4,86,61,US Senate,statewide,gen,False,Dennis DeConcini,democrat,False,total,400334,741210,False,20171011.0
3,1976,Arizona,AZ,4,86,61,US Senate,statewide,gen,False,Allan Norwitz,libertarian,False,total,7310,741210,False,20171011.0
4,1976,Arizona,AZ,4,86,61,US Senate,statewide,gen,False,Bob Field,independent,False,total,10765,741210,False,20171011.0


In [65]:
# Disregard special elections
df = df[df['special'] == False]
df = df[df.party.isin(['democrat', 'republican'])]
df.head()

,year,state,state_po,state_fips,state_cen,state_ic,office,district,stage,special,candidate,party,writein,mode,candidatevotes,totalvotes,unofficial,version
0,1976,Arizona,AZ,4,86,61,US Senate,statewide,gen,False,Sam Steiger,republican,False,total,321236,741210,False,20171011.0
2,1976,Arizona,AZ,4,86,61,US Senate,statewide,gen,False,Dennis DeConcini,democrat,False,total,400334,741210,False,20171011.0
6,1976,California,CA,6,93,71,US Senate,statewide,gen,False,S. I. (Sam) Hayakawa,republican,False,total,3748973,7470586,False,20171011.0
7,1976,California,CA,6,93,71,US Senate,statewide,gen,False,John V. Tunney,democrat,False,total,3502862,7470586,False,20171011.0
10,1976,Connecticut,CT,9,16,1,US Senate,statewide,gen,False,"Lowell P. Weicker, Jr.",republican,False,total,785683,1361666,False,20171011.0


In [66]:
# Combine all elections in state
df = df.groupby(['state', 'state_po', 'year', 'party'])[['candidatevotes', 'totalvotes']].sum().reset_index()
df

,state,state_po,year,party,candidatevotes,totalvotes
0,Alabama,AL,1978,democrat,547054,582005
1,Alabama,AL,1980,democrat,610175,1296757
2,Alabama,AL,1980,republican,650362,1296757
3,Alabama,AL,1984,democrat,860535,1371234
4,Alabama,AL,1984,republican,498508,1371234
...,...,...,...,...,...,...
1417,Wyoming,WY,2012,republican,185250,250700
1418,Wyoming,WY,2014,democrat,29377,171153
1419,Wyoming,WY,2014,republican,121554,171153
1420,Wyoming,WY,2018,democrat,61227,203420


In [67]:
# Get popular vote
df['pop_percent'] = df['candidatevotes'] / df['totalvotes']
df = df[['state', 'state_po', 'year', 'party', 'pop_percent']]
df

,state,state_po,year,party,pop_percent
0,Alabama,AL,1978,democrat,0.939947
1,Alabama,AL,1980,democrat,0.470539
2,Alabama,AL,1980,republican,0.501530
3,Alabama,AL,1984,democrat,0.627562
4,Alabama,AL,1984,republican,0.363547
...,...,...,...,...,...
1417,Wyoming,WY,2012,republican,0.738931
1418,Wyoming,WY,2014,democrat,0.171642
1419,Wyoming,WY,2014,republican,0.710207
1420,Wyoming,WY,2018,democrat,0.300988


In [68]:
# Reshape
df = df.pivot_table(index=['state', 'state_po', 'year'], columns=['party'], values='pop_percent').fillna(0).reset_index()
df

party,state,state_po,year,democrat,republican
0,Alabama,AL,1978,0.939947,0.000000
1,Alabama,AL,1980,0.470539,0.501530
2,Alabama,AL,1984,0.627562,0.363547
3,Alabama,AL,1986,0.502815,0.497185
4,Alabama,AL,1990,0.605671,0.394202
...,...,...,...,...,...
725,Wyoming,WY,2006,0.293917,0.688908
726,Wyoming,WY,2008,0.242517,0.756163
727,Wyoming,WY,2012,0.211484,0.738931
728,Wyoming,WY,2014,0.171642,0.710207


In [70]:
set(df['year'])

{1976,
 1978,
 1980,
 1982,
 1984,
 1986,
 1988,
 1990,
 1992,
 1994,
 1996,
 1998,
 2000,
 2002,
 2004,
 2006,
 2008,
 2010,
 2012,
 2014,
 2016,
 2018}